In [1]:
from torch import tensor

In [2]:
t1 = tensor([[1], [2], [3]])
t2 = tensor([1,2,3])
t1.shape, t2.shape

(torch.Size([3, 1]), torch.Size([3]))

In [3]:
t1[:, 0]

tensor([1, 2, 3])

In [4]:
# The forward and backward pass
def relu(x): return x.clamp_min(0.)

In [5]:
def lin(x, w, b): return x@w + b

In [6]:
import pickle, gzip, math, os, time, shutil, torch, matplotlib as mpl, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from torch import tensor, nn
import torch.nn.functional as F

In [7]:
from fastcore.test import test_close

torch.set_printoptions(precision=2, linewidth=2, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding = 'latin-1')
    x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

In [8]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

(torch.Size([50000, 784]),
 torch.Size([50000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

In [9]:
n, m = x_train.shape
c = y_train.max() + 1
n, m, c

(50000, 784, tensor(10))

In [10]:
nh = 50
w1 = torch.randn(m, nh)
b1 = torch.randn(nh)
w2 = torch.randn(nh, 1)
b2 = torch.randn(1)

In [11]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [12]:
t = relu(t)

In [13]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [14]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

#### Loss Function
ofcourse, `mse` is not suitable for multiclass classifier

In [15]:
res.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [16]:
(res.squeeze() - y_valid).shape

torch.Size([10000])

In [17]:
y_train, y_valid = y_train.float(), y_valid.float()

In [18]:
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [19]:
def mse(output, target):
    return (output[:, 0]-target).pow(2).mean()

In [20]:
mse(preds, y_train)

tensor(1099.57)

#### Gradient and backward pass

In [21]:
from sympy import symbols, diff
x, y = symbols('x y')
diff(x**2, x)

2*x

In [22]:
diff(3*x**2+9, x)

6*x

In [23]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    # import pdb; pdb.set_trace()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [24]:
def forward_and_backward(inp, target):
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:, 0] - target
    loss = diff.pow(2).mean()
    
    # backward pass
    out.g = 2*diff[:, None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [25]:
forward_and_backward(x_train, y_train)

In [26]:
x_train.g.shape # This is input gradients :)

torch.Size([50000, 784])

In [27]:
def get_grad(x): return x.g.clone()
chks = w1, w2, b1, b2, x_train
grads = w1g, w2g, b1g, b2g, ig = tuple(map(get_grad, chks))

In [28]:
# Using pytorch's auto grad 
def mkgrad(x):
    return x.clone().requires_grad_()
ptgrads = w12, w22, b12, b22, xt2 = tuple(map(mkgrad, chks))

In [29]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [30]:
loss = forward(xt2, y_train)
loss.backward()

In [31]:
xt2.grad.shape

torch.Size([50000, 784])

In [32]:
for a,b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

### Refactor Model
#### Layer as classes


In [33]:
class ReLU():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [34]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b
    def __call__(self, inp):
        self.inp = inp
        self.out = lin(self.inp, self.w, self.b)
        return self.out
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum()

In [40]:
class MSE():
    def __call__(self, inp, target):
        self.inp, self.target = inp, target
        self.out = mse(inp, target)
        return self.out
    def backward(self):
        self.inp.g = 2*(self.inp.squeeze() - self.target).unsqueeze(-1) / self.target.shape[0]

In [41]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), ReLU(), Lin(w2, b2)]
        self.loss = MSE()
    def __call__(self, x, target):
        for l in self.layers: x = l(x)
        return self.loss(x, target)
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [42]:
model = Model(w1, b1, w2, b2)

In [43]:

loss = model(x_train, y_train)

In [44]:
model.backward()

In [45]:
loss

tensor(1099.57)

In [46]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

AssertionError: close:
tensor([    -2.26,
             4.60,
            -1.04,
            15.45,
             1.46,
             1.06,
             0.30,
            -2.27,
            -4.57,
            27.46,
             1.50,
            -0.18,
             4.44,
            -1.66,
            -0.62,
             1.90,
            -0.35,
             3.02,
           -10.52,
            -1.99,
            -4.79,
            -0.25,
             1.53,
             5.97,
            -2.83,
             3.00,
             5.72,
             0.25,
            13.75,
             2.26,
            -2.78,
            11.07,
             1.06,
             8.98,
             0.35,
             2.28,
            -0.05,
             2.79,
            -1.92,
             5.67,
            -2.24,
            -0.02,
             0.78,
            -3.99,
             0.33,
             5.27,
             7.08,
             9.02,
            -8.80,
             5.53])
100.7018814086914

## Module.forward()
Module is a parent/core class which would be inherited and modified by child classes


In [48]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [54]:
class ReLU(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [50]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [51]:
class MSE(Module):
    def forward(self, inp, targ): return (inp.squeeze()-targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2* (inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [55]:
model = Model(w1, b1, w2, b2)

In [56]:
loss = model(x_train, y_train)

In [57]:
model.backward()

In [58]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

## Autograd

In [59]:
from torch import nn
import torch.nn.functional as F

In [66]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in, n_out).requires_grad_()
        self.b = torch.randn(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [67]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in, nh), nn.ReLU(), Linear(nh, n_out)]
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:, None])
    

In [68]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()


In [70]:
l0 = model.layers[0]
l0.b.grad.shape

torch.Size([50])